# DATA MINING - WEEK 4
## NGUYEN XUAN VIET DUC - 22280012
### Lesson 4: Association Rule Mining

### I. Objectives:
- Apriori algorithm

In [3]:
!pip install mlxtend

## II. Practice content

In [4]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
from itertools import combinations
from mlxtend.frequent_patterns import apriori as mlxtend_apriori
from mlxtend.frequent_patterns import association_rules as mlxtend_association_rules
from mlxtend.preprocessing import TransactionEncoder

# Load dữ liệu
df = pd.read_csv('dataW4.csv', header=None)
print(df)

       0      1       2       3     4      5
0   Wine  Chips  Bread   Butter  Milk  Apple
1   Wine    NaN  Bread   Butter  Milk    NaN
2    NaN    NaN  Bread   Butter  Milk    NaN
3    NaN  Chips     NaN     NaN   NaN  Apple
4   Wine  Chips  Bread   Butter  Milk  Apple
5   Wine  Chips     NaN     NaN  Milk    NaN
6   Wine  Chips  Bread   Butter   NaN  Apple
7   Wine  Chips     NaN     NaN  Milk    NaN
8   Wine    NaN  Bread      NaN   NaN  Apple
9   Wine    NaN  Bread   Butter  Milk    NaN
10   NaN  Chips  Bread   Butter   NaN  Apple
11  Wine    NaN     NaN  Butter  Milk  Apple
12  Wine  Chips  Bread   Butter  Milk    NaN
13  Wine    NaN  Bread      NaN  Milk  Apple
14  Wine    NaN  Bread   Butter  Milk  Apple
15  Wine  Chips  Bread   Butter  Milk  Apple
16   NaN  Chips  Bread   Butter  Milk  Apple
17   NaN  Chips     NaN  Butter  Milk  Apple
18  Wine  Chips  Bread   Butter  Milk  Apple
19  Wine    NaN  Bread   Butter  Milk  Apple
20  Wine  Chips  Bread      NaN  Milk  Apple
21   NaN  

In [5]:
data = df.copy()

records = []
for i in range(0, data.shape[0]):
    records.append([str(data.values[i,j]) for j in range(0, data.shape[1])])

In [6]:
te = TransactionEncoder()
te_ary = te.fit(records).transform(records)
df1 = pd.DataFrame(te_ary, columns=te.columns_)
print(df1)

    Apple  Bread   Butter  Chips   Milk   Wine    nan
0    True    True    True   True   True   True  False
1   False    True    True  False   True   True   True
2   False    True    True  False   True  False   True
3    True   False   False   True  False  False   True
4    True    True    True   True   True   True  False
5   False   False   False   True   True   True   True
6    True    True    True   True  False   True   True
7   False   False   False   True   True   True   True
8    True    True   False  False  False   True   True
9   False    True    True  False   True   True   True
10   True    True    True   True  False  False   True
11   True   False    True  False   True   True   True
12  False    True    True   True   True   True   True
13   True    True   False  False   True   True   True
14   True    True    True  False   True   True   True
15   True    True    True   True   True   True  False
16   True    True    True   True   True  False   True
17   True   False    True   

In [7]:
frequent_itemsets = apriori(df1, min_support=0.6, use_colnames=True)
print(frequent_itemsets)

    support      itemsets
0  0.681818       (Apple)
1  0.727273      (Bread )
2  0.681818      (Butter)
3  0.636364       (Chips)
4  0.772727        (Milk)
5  0.727273        (Wine)
6  0.818182         (nan)
7  0.636364  (Wine, Milk)


In [18]:
rules = association_rules(frequent_itemsets, metric='support', support_only=True, min_threshold=0.1)
rules = rules[['antecedents', 'consequents', 'support']]
print(rules)

  antecedents consequents   support
0      (Wine)      (Milk)  0.636364
1      (Milk)      (Wine)  0.636364


In [20]:

class AprioriAlgorithm:
    def __init__(self, min_support=0.5, min_confidence=0.7):
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.itemsets = {}
        self.rules = []
        self.item_counts = {}
        self.itemset_counts = {} 
    
    def fit(self, transactions):

        self.transactions = transactions
        self.n_transactions = len(transactions)
        
        # Find frequent 1-itemsets
        self._find_frequent_1_itemsets()
        self.print_frequent_1_itemsets()
        
        k = 2
        while True:
            if k-1 not in self.itemsets or not self.itemsets[k-1]:
                break
                
            candidate_itemsets = self._generate_candidate_itemsets(k)
            if not candidate_itemsets:
                break
                
            self.print_candidate_itemsets(k, candidate_itemsets)
                
            self.itemset_counts[k] = self._count_itemsets(candidate_itemsets)
            
            self.itemsets[k] = {itemset: count / self.n_transactions 
                               for itemset, count in self.itemset_counts[k].items() 
                               if count / self.n_transactions >= self.min_support}
            
            self.print_frequent_k_itemsets(k)
            
            k += 1
            
        self._generate_rules()
        self.print_rules()
        
        return self
    
    def _find_frequent_1_itemsets(self):
        self.item_counts = {}
        
        for transaction in self.transactions:
            for item in transaction:
                if item in self.item_counts:
                    self.item_counts[item] += 1
                else:
                    self.item_counts[item] = 1
        
        self.itemsets[1] = {frozenset([item]): count / self.n_transactions 
                           for item, count in self.item_counts.items() 
                           if count / self.n_transactions >= self.min_support}
    
    def _generate_candidate_itemsets(self, k):
        
        candidates = set()
        prev_itemsets = list(self.itemsets[k-1].keys())
        
        for i in range(len(prev_itemsets)):
            for j in range(i+1, len(prev_itemsets)):
                itemset1 = sorted(list(prev_itemsets[i]))
                itemset2 = sorted(list(prev_itemsets[j]))

                if k == 2:
                    new_itemset = frozenset(itemset1 + itemset2)
                    if len(new_itemset) == k:
                        candidates.add(new_itemset)
                else:
                    if itemset1[:-1] == itemset2[:-1]:
                        new_itemset = frozenset(itemset1 + [itemset2[-1]])

                        if self._all_subsets_frequent(new_itemset, k-1):
                            candidates.add(new_itemset)
        
        return candidates
    
    def _all_subsets_frequent(self, itemset, k):
        
        for subset in combinations(itemset, k):
            if frozenset(subset) not in self.itemsets[k]:
                return False
        return True
    
    def _count_itemsets(self, candidate_itemsets):

        itemset_counts = {itemset: 0 for itemset in candidate_itemsets}
        
        for transaction in self.transactions:
            transaction_set = set(transaction)
            for itemset in candidate_itemsets:
                if all(item in transaction_set for item in itemset):
                    itemset_counts[itemset] += 1
                    
        return itemset_counts
    
    def _generate_rules(self):

        self.rules = []
        
        for k in range(2, len(self.itemsets) + 1):
            if k not in self.itemsets:
                continue
                
            for itemset, support in self.itemsets[k].items():
                for i in range(1, k):
                    for antecedent in combinations(itemset, i):
                        antecedent = frozenset(antecedent)
                        consequent = frozenset(itemset - antecedent)
                        
                        confidence = support / self.itemsets[len(antecedent)][antecedent]
                        
                        if confidence >= self.min_confidence:
                            self.rules.append({
                                'antecedent': list(antecedent),
                                'consequent': list(consequent),
                                'support': support,
                                'confidence': confidence
                            })
    
    def get_itemsets(self):
        result = []
        for k in self.itemsets:
            for itemset, support in self.itemsets[k].items():
                result.append({
                    'itemset': list(itemset),
                    'support': support
                })
        return result
    
    def get_rules(self):
        return self.rules
    

    def print_frequent_1_itemsets(self):
        print("\n- Với k = 1, ta khởi tạo F1")
        print("Item\t\tfrequency\tsupport")
        for item, count in sorted(self.item_counts.items()):
            if count / self.n_transactions >= self.min_support:
                support_pct = count / self.n_transactions * 100
                print(f"{item}\t\t{count}\t\t{support_pct:.0f}%")
    
    def print_candidate_itemsets(self, k, candidates):
        print(f"\n- Khởi tạo C{k} bằng việc kết hợp các cặp item của F{k-1}")
        candidate_str = ", ".join([str(list(c)) for c in candidates])
        print(f"{candidate_str}")
    
    def print_frequent_k_itemsets(self, k):
        print(f"\n- Tạo F{k}")
        print("Item\t\t\tfrequency\tsupport")
        
        for itemset in sorted(self.itemset_counts[k].keys(), key=lambda x: tuple(sorted(x))):
            count = self.itemset_counts[k][itemset]
            support_pct = count / self.n_transactions * 100
            items_str = ", ".join(sorted(itemset))
            print(f"{items_str}\t\t{count}\t\t{support_pct:.0f}%")
        
        print(f"\n- Tìm các hạng mục quan trọng dựa vào minsup = {self.min_support*100:.0f}% nên ta lấy các {k}-item sau:")
        important_itemsets = [list(itemset) for itemset in self.itemsets[k].keys()]
        if important_itemsets:
            print(f"{important_itemsets}")
        else:
            print(f"F{k} = ∅")
    
    def print_rules(self):
        print("\n- Phát sinh các luật")
        for rule in self.rules:
            antecedent = ", ".join(rule['antecedent'])
            consequent = ", ".join(rule['consequent'])
            support = rule['support']
            confidence = rule['confidence']
            
            print(f"{antecedent} → {consequent} có conf({antecedent} → {consequent}) = support({antecedent}, {consequent})/support({antecedent}) = {confidence*100:.0f}%")


In [22]:
def load_data_from_csv(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    transactions = []
    for _, row in df.iterrows():
        # Filter out NaN values and convert to list
        transaction = [item for item in row if pd.notna(item)]
        transactions.append(transaction)
    
    print(f"Loaded {len(transactions)} transactions from {file_path}")
    return transactions


def run_mlxtend_apriori(transactions, min_support=0.6, min_confidence=0.8):

    start_time = time.time()
    
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    
    # Find frequent itemsets
    frequent_itemsets = mlxtend_apriori(df, min_support=min_support, use_colnames=True)
    
    # Generate rules
    rules = mlxtend_association_rules(frequent_itemsets, metric="confidence", 
                                     min_threshold=min_confidence)
    
    execution_time = time.time() - start_time
    
    return frequent_itemsets, rules, execution_time


def compare_implementations(transactions, min_support=0.6, min_confidence=0.8):

    print("\n" + "="*50)
    print("SO SÁNH IMPLEMENTATION TỰ VIẾT VỚI THƯ VIỆN MLXTEND")
    print("="*50)
    
    print("\n1. IMPLEMENTATION TỰ VIẾT")
    custom_start_time = time.time()
    custom_apriori = AprioriAlgorithm(min_support=min_support, min_confidence=min_confidence)
    custom_apriori.fit(transactions)
    custom_time = time.time() - custom_start_time
    
    custom_itemsets = custom_apriori.get_itemsets()
    custom_rules = custom_apriori.get_rules()
    
    print("\n2. IMPLEMENTATION THƯ VIỆN MLXTEND")
    mlxtend_itemsets, mlxtend_rules, mlxtend_time = run_mlxtend_apriori(
        transactions, min_support, min_confidence
    )
    
    mlxtend_itemsets_converted = []
    for _, row in mlxtend_itemsets.iterrows():
        itemset = list(row['itemsets'])
        support = row['support']
        mlxtend_itemsets_converted.append({
            'itemset': itemset,
            'support': support
        })
    
    mlxtend_rules_converted = []
    for _, row in mlxtend_rules.iterrows():
        mlxtend_rules_converted.append({
            'antecedent': list(row['antecedents']),
            'consequent': list(row['consequents']),
            'support': row['support'],
            'confidence': row['confidence']
        })
    
    print("\n3. KẾT QUẢ SO SÁNH")
    print(f"- Thời gian thực thi:")
    print(f"  - Implementation tự viết: {custom_time:.4f} giây")
    print(f"  - Thư viện mlxtend: {mlxtend_time:.4f} giây")
    
    print(f"\n- Số lượng itemsets:")
    print(f"  - Implementation tự viết: {len(custom_itemsets)}")
    print(f"  - Thư viện mlxtend: {len(mlxtend_itemsets_converted)}")
    
    print(f"\n- Số lượng luật:")
    print(f"  - Implementation tự viết: {len(custom_rules)}")
    print(f"  - Thư viện mlxtend: {len(mlxtend_rules_converted)}")
    
    # Check if results match
    itemsets_match = len(custom_itemsets) == len(mlxtend_itemsets_converted)
    rules_match = len(custom_rules) == len(mlxtend_rules_converted)
    
    print(f"\n- Kết quả có trùng khớp không:")
    print(f"  - Itemsets: {'Có' if itemsets_match else 'Không'}")
    print(f"  - Luật: {'Có' if rules_match else 'Không'}")
    
    print("\n4. MẪU CÁC ITEMSETS TÌM ĐƯỢC")
    print("- Implementation tự viết (top 5):")
    for i, item in enumerate(sorted(custom_itemsets, key=lambda x: x['support'], reverse=True)[:5]):
        print(f"  {i+1}. {item['itemset']} (support: {item['support']:.4f})")
    
    print("\n- Thư viện mlxtend (top 5):")
    for i, item in enumerate(sorted(mlxtend_itemsets_converted, key=lambda x: x['support'], reverse=True)[:5]):
        print(f"  {i+1}. {item['itemset']} (support: {item['support']:.4f})")
    
    print("\n5. MẪU CÁC LUẬT KẾT HỢP TÌM ĐƯỢC")
    print("- Implementation tự viết (top 5):")
    for i, rule in enumerate(sorted(custom_rules, key=lambda x: x['confidence'], reverse=True)[:5]):
        print(f"  {i+1}. {rule['antecedent']} → {rule['consequent']} "
              f"(support: {rule['support']:.4f}, confidence: {rule['confidence']:.4f})")
    
    print("\n- Thư viện mlxtend (top 5):")
    for i, rule in enumerate(sorted(mlxtend_rules_converted, key=lambda x: x['confidence'], reverse=True)[:5]):
        print(f"  {i+1}. {rule['antecedent']} → {rule['consequent']} "
              f"(support: {rule['support']:.4f}, confidence: {rule['confidence']:.4f})")
    
    print("\n6. KẾT LUẬN")
    if itemsets_match and rules_match:
        print("- Implementation tự viết cho kết quả trùng khớp với thư viện mlxtend")
        # Fix the division by zero error
        if custom_time <= 0 or mlxtend_time <= 0:
            print("- Thời gian thực thi quá nhanh để so sánh chính xác")
        elif custom_time < mlxtend_time:
            print(f"- Implementation tự viết nhanh hơn {(mlxtend_time/custom_time):.2f} lần so với thư viện mlxtend")
        else:
            print(f"- Thư viện mlxtend nhanh hơn {(custom_time/mlxtend_time):.2f} lần so với implementation tự viết")
    else:
        print("- Implementation tự viết cho kết quả khác với thư viện mlxtend")
        print("- Cần kiểm tra lại thuật toán hoặc các tham số đầu vào")


def main():
    
    csv_file = "dataW4.csv"
    transactions = load_data_from_csv(csv_file)
    
    print("Sample of transactions:")
    for i, transaction in enumerate(transactions[:5]):
        print(f"Transaction {i+1}: {transaction}")
    
    min_support = 0.6
    min_confidence = 0.8
    
    print(f"\nRunning Apriori with min_support={min_support*100:.0f}% and min_confidence={min_confidence*100:.0f}%")
    
    apriori = AprioriAlgorithm(min_support=min_support, min_confidence=min_confidence)
    apriori.fit(transactions)

    compare_implementations(transactions, min_support, min_confidence)

    print("\n=== Summary of Results ===")
    print(f"Total transactions processed: {len(transactions)}")
    print(f"Total frequent itemsets found: {sum(len(itemsets) for itemsets in apriori.itemsets.values())}")
    print(f"Total association rules found: {len(apriori.rules)}")

if __name__ == "__main__":
    main()

Loaded 21 transactions from dataW4.csv
Sample of transactions:
Transaction 1: ['Wine', 'Bread ', 'Butter', 'Milk']
Transaction 2: ['Bread ', 'Butter', 'Milk']
Transaction 3: ['Chips', 'Apple']
Transaction 4: ['Wine', 'Chips', 'Bread ', 'Butter', 'Milk', 'Apple']
Transaction 5: ['Wine', 'Chips', 'Milk']

Running Apriori with min_support=60% and min_confidence=80%

- Với k = 1, ta khởi tạo F1
Item		frequency	support
Apple		14		67%
Bread 		15		71%
Butter		14		67%
Chips		13		62%
Milk		16		76%
Wine		15		71%

- Khởi tạo C2 bằng việc kết hợp các cặp item của F1
['Milk', 'Apple'], ['Butter', 'Milk'], ['Butter', 'Chips'], ['Wine', 'Chips'], ['Bread ', 'Milk'], ['Bread ', 'Chips'], ['Wine', 'Milk'], ['Bread ', 'Apple'], ['Chips', 'Apple'], ['Wine', 'Apple'], ['Butter', 'Bread '], ['Butter', 'Apple'], ['Chips', 'Milk'], ['Bread ', 'Wine'], ['Butter', 'Wine']

- Tạo F2
Item			frequency	support
Apple, Bread 		11		52%
Apple, Butter		10		48%
Apple, Chips		9		43%
Apple, Milk		10		48%
Apple, Wine		10		